## Data Preparation

The code which follows will be using the data exported from the ASL_Preprocessing phase to train a model. This will be a classification model and will make use of neural network.

In [34]:
import pandas as pd

In [35]:
# let's mount the google drive with the landmark information.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
# Let's set up some constants we will be using from hear on out
csv_path = '/content/drive/MyDrive/landmarks.csv'

In [37]:
df = pd.read_csv(csv_path)

In [38]:
df.head()

,1_X,1_Y,2_X,2_Y,3_X,3_Y,4_X,4_Y,5_X,5_Y,...,17_Y,18_X,18_Y,19_X,19_Y,20_X,20_Y,21_X,21_Y,Label
0,0.250000,1.0,0.705882,0.899281,1.000000,0.690647,0.823529,0.474820,0.485294,0.402878,...,0.654676,0.000000,0.460432,0.058824,0.266187,0.102941,0.129496,0.147059,0.000000,I
1,0.278689,1.0,0.721311,0.926667,1.000000,0.753333,0.819672,0.606667,0.475410,0.580000,...,0.046667,0.000000,0.553333,0.016393,0.386667,0.016393,0.286667,0.032787,0.193333,B
2,0.669565,1.0,0.834783,0.717647,0.965217,0.447059,1.000000,0.188235,0.991304,0.000000,...,0.670588,0.330435,0.623529,0.182609,0.447059,0.086957,0.341176,0.000000,0.235294,Y
3,0.107143,1.0,0.375000,1.000000,0.625000,0.887218,0.839286,0.781955,1.000000,0.706767,...,0.789474,0.017857,0.601504,0.008929,0.503759,0.000000,0.661654,0.008929,0.774436,L
4,0.383333,1.0,0.816667,0.875000,1.000000,0.487500,0.666667,0.212500,0.283333,0.275000,...,0.575000,0.000000,0.462500,0.000000,0.325000,0.100000,0.525000,0.100000,0.612500,S


In [39]:
from sklearn.preprocessing import OneHotEncoder
# Transform all the labels into dummy variables
encoder = OneHotEncoder(sparse_output=False)
labels  = df[['Label']].values
ohe_labels = encoder.fit_transform(labels)

# Retreive the headers
headers = encoder.get_feature_names_out(['Label'])
ohe_df  = pd.DataFrame(ohe_labels)


ohe_df.columns = headers
ohe_df.head()

,Label_A,Label_B,Label_C,Label_D,Label_E,Label_F,Label_G,Label_H,Label_I,Label_J,...,Label_T,Label_U,Label_V,Label_W,Label_X,Label_Y,Label_Z,Label_d,Label_n,Label_s
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
from sklearn.model_selection import train_test_split

X = df.drop('Label', axis=1)
y = ohe_df

print(type(X))
print(type(y))
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.25)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


## Model Testing and Selection

We have 21 data points each with two coordinates - X and Y. Therefore, we have 42 inputs. The output will be 1 of 29 possible outputs, i.e. 26 alphabetical outputs and 3 control charachters. Consequently, we will need a softmax of 29 neurons.

I plan to experiment with a few different dense layer configurations to test which gives the best peroformance. I will be using relu as the main activation layer except the final layer which is a softmax.

|Model|Configuration|Reasoning|
|-------|-------|--------|
|model1|Input(42), Softmax(29)| Test the ANN at its most basic i.e. input and output minimums
|model2|Input(42), Dense(21), Dense(10), Softmax(29)|Ue an autoencoder architecture to reduce the number of neurons to a code of 10 and then inflate the number of neurons in the output
|model3|Input(42), Dense(84), Softmax(29)| Increase the number of neurons for more data representation|


In [41]:
from keras import Sequential
from keras.layers import Input, Dense, Softmax
from sklearn.metrics import f1_score
import numpy as np

In [42]:
model1 = Sequential()

model1.add(Input(shape=(42,)))
model1.add(Dense(29, activation='softmax'))
model1.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 29)                  │           1,247 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,247 (4.87 KB)

 Trainable params: 1,247 (4.87 KB)

 Non-trainable params: 0 (0.00 B)

In [43]:
model2 = Sequential()

model2.add(Input(shape=(42,)))
model2.add(Dense(21, activation='relu'))
model2.add(Dense(10, activation='relu'))
model2.add(Dense(29, activation='softmax'))
model2.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                      │ (None, 21)                  │             903 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 10)                  │             220 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 29)                  │             319 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,442 (5.63 KB)

 Trainable params: 1,442 (5.63 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
model3 = Sequential()

model3.add(Input(shape=(42,)))
model3.add(Dense(84, activation='relu'))
model3.add(Dense(29, activation='softmax'))
model3.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                     │ (None, 84)                  │           3,612 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 29)                  │           2,465 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,077 (23.74 KB)

 Trainable params: 6,077 (23.74 KB)

 Non-trainable params: 0 (0.00 B)

#### Model 1 Testing

In [45]:
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
model1.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5948 - loss: 2.2372
Epoch 2/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9268 - loss: 0.6940
Epoch 3/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9415 - loss: 0.4364
Epoch 4/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9494 - loss: 0.3427
Epoch 5/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9568 - loss: 0.2891
Epoch 6/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9586 - loss: 0.2613
Epoch 7/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9625 - loss: 0.2381
Epoch 8/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9627 - loss: 0.2267
Epoch 9/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9657 - loss: 0.2114
Epoch 10/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9665 - loss: 0.2015
Epoch 11/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9692 - loss: 0.1939
Epoch 12/20
2310/2310 ━━━━━━━━

In [47]:
model1_loss, model1_accuracy = model1.evaluate(X_val, y_val)
print('Test loss', model1_loss)
print('Accuracy', model1_accuracy)

770/770 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9759 - loss: 0.1464
Test loss 0.15359848737716675
Accuracy 0.9748305082321167


In [48]:
y_pred = model1.predict(X_test)
y_inv_pred = np.argmax(y_pred, axis=1)
y_inv_test = np.argmax(y_test, axis=1)

model1_f1 = f1_score(y_inv_test, y_inv_pred, average='micro')
print('F1 score ', model1_f1 )



1027/1027 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
F1 score  0.9754901960784313


#### Model 2 Testing

In [49]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [50]:
model2.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.5562 - loss: 1.7349
Epoch 2/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9541 - loss: 0.2646
Epoch 3/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9620 - loss: 0.1969
Epoch 4/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9666 - loss: 0.1642
Epoch 5/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9702 - loss: 0.1477
Epoch 6/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9719 - loss: 0.1433
Epoch 7/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9735 - loss: 0.1298
Epoch 8/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9739 - loss: 0.1263
Epoch 9/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9746 - loss: 0.1195
Epoch 10/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9766 - loss: 0.1103
Epoch 11/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9777 - loss: 0.1074
Epoch 12/20
2310/2310 ━━━━━━━━

In [51]:
model2_loss, model2_accuracy = model2.evaluate(X_val, y_val)
print('Test loss', model2_loss)
print('Accuracy', model2_accuracy)

770/770 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9828 - loss: 0.0788
Test loss 0.08636102825403214
Accuracy 0.9815288186073303


In [52]:
y_pred = model2.predict(X_test)
y_inv_pred = np.argmax(y_pred, axis=1)
y_inv_test = np.argmax(y_test, axis=1)

model2_f1 = f1_score(y_inv_test, y_inv_pred, average='micro')
print('F1 score ', model2_f1 )


1027/1027 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
F1 score  0.9814273535501157


#### Model 3 Testing

In [53]:
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [54]:
model3.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.7507 - loss: 1.2355
Epoch 2/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9639 - loss: 0.1892
Epoch 3/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9718 - loss: 0.1368
Epoch 4/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9755 - loss: 0.1185
Epoch 5/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9783 - loss: 0.1030
Epoch 6/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9802 - loss: 0.0917
Epoch 7/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9806 - loss: 0.0852
Epoch 8/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9830 - loss: 0.0785
Epoch 9/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9825 - loss: 0.0761
Epoch 10/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9840 - loss: 0.0705
Epoch 11/20
2310/2310 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9854 - loss: 0.0623
Epoch 12/20
2310/2310 ━━━━━━━━

In [55]:
model3_loss, model3_accuracy = model3.evaluate(X_val, y_val)
print('Test loss', model3_loss)
print('Accuracy', model3_accuracy)

770/770 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9897 - loss: 0.0473
Test loss 0.052469752728939056
Accuracy 0.988714337348938


In [56]:
y_pred = model3.predict(X_test)
y_inv_pred = np.argmax(y_pred, axis=1)
y_inv_test = np.argmax(y_test, axis=1)

model3_f1 = f1_score(y_inv_test, y_inv_pred, average='micro')
print('F1 score ', model3_f1 )


1027/1027 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
F1 score  0.9889477530142492


### Summary

From the results, model3 provides the best model performance of the three confiurations.

|Model|Loss|Accuracy|F1|
|----|---|---|---|
|model1|15%|97.48%|97.55%|
|model2|9%|98.15%|98.14%|
|model3|5%|98.87|98.90%|

In [57]:
model3.save('/content/drive/MyDrive/asl_model.keras')